In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_wine
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import (cross_val_score, KFold, cross_validate,
                                     train_test_split)
from sklearn.ensemble import StackingClassifier

In [2]:
data = load_wine()
y = data.target
X = data.data
stc = StandardScaler()
lenc = LabelEncoder()
columns = data.feature_names
df = pd.DataFrame(data=np.hstack(tup=(X, y.reshape(-1, 1))),
                  columns=np.hstack(tup=(columns, ["Class"])))
X_std = stc.fit_transform(df[columns])

In [3]:
pipesvm = Pipeline([("stc", stc), ("selection", RFE(LinearSVC())),
                    ("svm", SVC(kernel="linear"))])
pipelda = Pipeline([("stc", stc), ("svm", LinearDiscriminantAnalysis())])
estimators = [("LDA", pipelda), ("SVM", pipesvm)]
# El utilizar clasificadores apilados tiene beneficios cuando se trata de
# problemas multiclase, puesto que puede mejorar mucho el pronostico de clase
# al explotar el poder predictivo del pronostico para ciertas clases
stacking_classifier = StackingClassifier(estimators=estimators,
                                         final_estimator=GaussianNB())

In [4]:
print("Stacking stimators")
print(cross_val_score(X=df[columns], y=y, estimator=stacking_classifier,
                      cv=KFold(5)))

Stacking stimators
[1.         0.88888889 0.97222222 0.91428571 1.        ]


In [5]:
print("Only SVM")
print(cross_val_score(X=df[columns], y=y, estimator=pipesvm, cv=KFold(5)))

Only SVM
[0.97222222 0.86111111 0.91666667 0.91428571 1.        ]


In [6]:
print("Only LDA")
print(cross_val_score(X=df[columns], y=y, estimator=pipelda, cv=KFold(5)))

Only LDA
[0.94444444 0.88888889 0.94444444 0.94285714 1.        ]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df[columns], y,\
    test_size=0.33, random_state=42)
stacking_classifier.fit(X_train, y_train)
y_pred = stacking_classifier.predict(X_test)
print("\n")
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))



[[20  0  0]
 [ 0 24  0]
 [ 0  0 15]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        24
           2       1.00      1.00      1.00        15

    accuracy                           1.00        59
   macro avg       1.00      1.00      1.00        59
weighted avg       1.00      1.00      1.00        59

